<a href="https://colab.research.google.com/github/JhonatanWalterSen/spark-in-colab/blob/main/Almacenamiento_en_cache.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Almacenamiento en Caché

# Instalaciones

In [1]:
# Instalar Java 8 scas
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
# Descargar Spark
!wget -q http://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

In [3]:
# Descomprimir Spart
!tar xf spark-3.3.1-bin-hadoop3.tgz

In [4]:
# Establecer las variables de Entorno
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [5]:
# Instalar FindSpark en el sistema
!pip install -q findspark

In [6]:
import findspark

In [7]:
findspark.init()

In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Almacenamiento en cahé').master('local[*]').getOrCreate()

In [13]:
spark


In [14]:
sc = spark.sparkContext

# Almacenamiento

In [15]:
rdd = sc.parallelize([item for item in range(10)])

In [16]:
rdd.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [17]:
from pyspark.storagelevel import StorageLevel

In [18]:
rdd.persist(StorageLevel.MEMORY_ONLY)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [19]:
rdd.unpersist()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [20]:
rdd.persist(StorageLevel.DISK_ONLY)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [21]:
rdd.unpersist()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [22]:
rdd.cache()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

# Particiones

In [23]:
num_particiones = 6

In [24]:
rdd = sc.parallelize(['x','y','z'])

In [25]:
# indice = hash(item) % num_particiones

In [26]:
hash('x') % num_particiones

5

In [27]:
hash('y') % num_particiones

1

In [28]:
hash('z') % num_particiones

0

# Variables BroadCast

In [48]:
rdd_datos = sc.parallelize([item for item in range(10)])

In [50]:
uno = 1

In [51]:
br_uno = sc.broadcast(uno)

In [44]:
rdd1 = rdd_datos.map(lambda x: x + br_uno.value)

In [52]:
rdd1.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
#destruir broadcast con .destroy()

# Acumuladores

In [53]:
acumulador = sc.accumulator(0)

In [54]:
rdd_ac = sc.parallelize([2,4,6,8,10])

In [55]:
rdd_ac.foreach(lambda x : acumulador.add(x))

In [56]:
print(acumulador.value)

30


In [57]:
rdd1 = sc.parallelize('hola me llamo walter xd'.split(' '))

In [58]:
acumulador1 = sc.accumulator(0)

In [59]:
rdd1.foreach(lambda x: acumulador1.add(1))

In [60]:
print(acumulador1.value)

5


# Ejercicios


1.   Cree un RDD importes a partir de los datos adjuntos a esta lección como recurso. Emplee acumuladores para obtener el total de ventas realizadas y el importe total de las ventas.

2.   Si se conoce que a cada venta hay que restarle un importe fijo igual a 10 pesos por temas de impuestos.

  a ¿Cómo restaría este impuesto de cada venta utilizando una variable broadcast para acelerar el proceso?

  b Cree un RDD llamado ventas_sin_impuestos a partir de la propuesta del inciso a que contenga las ventas sin impuestos.
  c Destruya la variable broadcast creada luego de emplearla para crear el RDD del inciso b.

3.   Persista el RDD ventas_sin_impuestos en los siguientes niveles de persistencia.

  a Memoria.

  b Disco solamente

  c Memoria y disco.

In [61]:
importes = sc.textFile('./rdd.txt')

In [62]:
importes.take(5)

['527', '386', '701', '240', '941']

In [63]:
total_ventas = sc.accumulator(0)
importe_total = sc.accumulator(0)

In [64]:
importes.foreach(lambda x: total_ventas.add(1))

importes.foreach(lambda x: importe_total.add(float(x)))

In [65]:
total_ventas.value

10000

In [67]:
importe_total.value

5042335.0

2

In [68]:
impuesto = sc.broadcast(10)

In [69]:
ventas_sin_impuestos = importes.map(lambda x: float(x) - impuesto.value)

In [70]:
ventas_sin_impuestos.take(5)

[517.0, 376.0, 691.0, 230.0, 931.0]

In [71]:
impuesto.destroy()

3

In [72]:
from pyspark.storagelevel import StorageLevel

In [73]:
ventas_sin_impuestos.cache()

PythonRDD[18] at RDD at PythonRDD.scala:53

In [74]:
ventas_sin_impuestos.unpersist()

PythonRDD[18] at RDD at PythonRDD.scala:53

In [75]:
ventas_sin_impuestos.persist(StorageLevel.DISK_ONLY)

PythonRDD[18] at RDD at PythonRDD.scala:53

In [76]:
ventas_sin_impuestos.unpersist()

PythonRDD[18] at RDD at PythonRDD.scala:53

In [77]:
ventas_sin_impuestos.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[18] at RDD at PythonRDD.scala:53

In [78]:
ventas_sin_impuestos.unpersist()

PythonRDD[18] at RDD at PythonRDD.scala:53